In [1]:
from tqdm import tqdm

import psycopg2
from sqlalchemy import make_url

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.vector_stores import VectorStoreQuery
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import TextNode

if __name__ == "__main__":
    db_name = "vector_db"
    host = "localhost"
    password = "12345678"
    port = "5432"
    user = "postgres"

    conn = psycopg2.connect(
        dbname="postgres",
        host=host,
        password=password,
        port=port,
        user=user,
    )
    print("Connect to DB Successful")
    conn.autocommit = True

    vector_store = PGVectorStore.from_params(
        database=db_name,
        host=host,
        password=password,
        port=port,
        user=user,
        table_name="test",
        embed_dim=768,  
    )

    embed_model = HuggingFaceEmbedding(model_name="bkai-foundation-models/vietnamese-bi-encoder")

    query_str = "Cần chạy xe tới khi đi đăng ký xe máy hay không?"
    
    query_embedding = embed_model.get_query_embedding(query_str)

    query_mode = "default"
    # query_mode = "sparse"
    # query_mode = "hybrid"

    vector_store_query = VectorStoreQuery(
        query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
    )

    query_result = vector_store.query(vector_store_query)
    print(query_result.nodes[0].get_content())

Connect to DB Sucessful
1. Hồ sơ đăng ký xe máy chuyên dùng lần đầu bao gồm:
a) Tờ khai đăng ký xe máy chuyên dùng theo mẫu số 2 của Phụ lục 2 ban hành kèm theo Thông tư này;
b) Bản chính giấy tờ xác định quyền sở hữu xe máy chuyên dùng theo quy định tại Điều 3 của Thông tư này trừ các trường hợp quy định tại khoản 3 Điều 19 của Thông tư này;
c) Bản chính giấy tờ xác định nguồn gốc xe máy chuyên dùng theo quy định tại Điều 4 của Thông tư này trừ các trường hợp quy định tại khoản 3 Điều 19 của Thông tư này.
2. Xe máy chuyên dùng được tạm nhập, tái xuất thuộc các chương trình dự án của nước ngoài đầu tư tại Việt Nam, các dự án ODA đầu tư tại Việt Nam phải thực hiện đăng ký có thời hạn. Hồ sơ đăng ký bao gồm:
a) Tờ khai đăng ký có thời hạn xe máy chuyên dùng theo mẫu số 3 của Phụ lục 2 ban hành kèm theo Thông tư này;
b) Giấy phép đầu tư hoặc Giấy chứng nhận đăng ký đầu tư (bản sao và xuất trình bản chính để đối chiếu hoặc bản sao có chứng thực);
c) Bản chính giấy tờ xác định quyền sở hữu 

In [8]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))